In [1]:
import numpy as np
p = 3000
n = int(p/2)
niter = 11
comp_time_fista = np.zeros((niter,6))

In [2]:
from ctypes import *
import numpy as np
import time


np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float32)
tr_beta = np.zeros(p).astype(np.float32)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float32)

libc_FISTA_dll = CDLL("/home/dyu/Dropbox/tf-notebooks/cuda_comp/fista_example/using_dll/fista_dll_float.so")
FISTA_Kernel_S = libc_FISTA_dll.FISTA_S

FISTA_Kernel_S.restype = None
FISTA_Kernel_S.argtypes = (POINTER(c_float), POINTER(c_float), POINTER(c_float), 
                         POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))



for i in np.arange(niter):
    beta = np.zeros(p).astype(np.float32)
    beta_cr = beta.ctypes.data_as(POINTER(c_float))
    X_cr =  X.ctypes.data_as(POINTER(c_float)) 
    y_cr = y.ctypes.data_as(POINTER(c_float))


    loss = np.zeros(5000).astype(np.float32)## max_iter size만큼
    loss_c = loss.ctypes.data_as(POINTER(c_float)) 
    lam =  np.sqrt(2 * np.log(p) / n)
    lam_c = c_float(lam)
    L_c = c_float(10)
    eta_c = c_float(2.0)
    tol_c = c_float(1e-03)

    iter_c = c_int(5000)
    n_c = c_int(n)
    p_c = c_int(p)
    step_c = c_int(0)

    t1 = time.time()
    FISTA_Kernel_S(beta_cr, X_cr, y_cr, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
    t2 = time.time()
    comp_time_fista[i,0] = t2-t1

    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,0])
    




  1 -th iteration, Collapsed Time:  1.7543652057647705

  2 -th iteration, Collapsed Time:  1.626410722732544

  3 -th iteration, Collapsed Time:  1.6284685134887695

  4 -th iteration, Collapsed Time:  1.628798246383667

  5 -th iteration, Collapsed Time:  1.6242821216583252

  6 -th iteration, Collapsed Time:  1.6260333061218262

  7 -th iteration, Collapsed Time:  1.632284164428711

  8 -th iteration, Collapsed Time:  1.6383018493652344

  9 -th iteration, Collapsed Time:  1.612287998199463

  10 -th iteration, Collapsed Time:  1.6460251808166504

  11 -th iteration, Collapsed Time:  1.6154115200042725


In [3]:
step_c

c_int(1746)

In [4]:
beta_cr[:5]

[0.9535295963287354,
 0.9764904379844666,
 0.9542718529701233,
 0.9487429857254028,
 1.050582766532898]

In [5]:
loss_c[:5]

[3.142929792404175,
 1.5433892011642456,
 1.2192375659942627,
 0.9664725065231323,
 0.7812541127204895]

In [6]:
from ctypes import *
import numpy as np
import time


np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float32)
tr_beta = np.zeros(p).astype(np.float32)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float32)

libc_FISTA_blas = CDLL("/home/dyu/Dropbox/tf-notebooks/cuda_comp/fista_example/using_dll/fista_cublas_float.so")
FISTA_blas = libc_FISTA_blas.FISTA

FISTA_blas.restype = None
FISTA_blas.argtypes = (POINTER(c_float), POINTER(c_float), POINTER(c_float), 
                         POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float), POINTER(c_float),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))

for i in np.arange(niter):
    
    beta = np.zeros(p).astype(np.float32)
    beta_cc = beta.ctypes.data_as(POINTER(c_float))
    X_c =  X.ravel(order = "F")              ## n*p 에 () 반드시 필요, c_double * dim 형태만 지원 
    X_cc = X_c.ctypes.data_as(POINTER(c_float))
    y_cc = y.ctypes.data_as(POINTER(c_float))

    loss = np.zeros(5000).astype(np.float32)## max_iter size만큼
    loss_c = loss.ctypes.data_as(POINTER(c_float)) 
    lam =  np.sqrt(2 * np.log(p) / n).astype(np.float32)
    lam_c = c_float(lam)
    L_c = c_float(10.0)
    eta_c = c_float(2.0)
    tol_c = c_float(1e-03)

    iter_c = c_int(5000)
    n_c = c_int(n)
    p_c = c_int(p)
    step_c = c_int(0)

    t1 = time.time()
    FISTA_blas(beta_cc, X_cc, y_cc, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
    t2 = time.time()
    comp_time_fista[i,1] = t2-t1

    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,1])
    



  1 -th iteration, Collapsed Time:  0.7872188091278076

  2 -th iteration, Collapsed Time:  0.4914968013763428

  3 -th iteration, Collapsed Time:  0.5087156295776367

  4 -th iteration, Collapsed Time:  0.4932122230529785

  5 -th iteration, Collapsed Time:  0.513617992401123

  6 -th iteration, Collapsed Time:  0.4919900894165039

  7 -th iteration, Collapsed Time:  0.5067856311798096

  8 -th iteration, Collapsed Time:  0.49361395835876465

  9 -th iteration, Collapsed Time:  0.4910733699798584

  10 -th iteration, Collapsed Time:  0.5302181243896484

  11 -th iteration, Collapsed Time:  0.4998292922973633


In [7]:
step_c

c_int(1743)

In [8]:
beta_cc[:5]

[0.9540179371833801,
 0.9741050601005554,
 0.9583085775375366,
 0.9486698508262634,
 1.0489410161972046]

In [9]:
loss_c[:5]

[3.142929792404175,
 1.5433893203735352,
 1.2192375659942627,
 0.9664724469184875,
 0.7812541127204895]

In [10]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray, tools, cumath
import numpy as np
import time
import math

with open('/home/dyu/Dropbox/tf-notebooks/cuda_comp/fista_example/pycuda/FISTA_kernel.cu') as f:
    FISTA_kernel = f.read()
    
FISTA_module = SourceModule(FISTA_kernel)

Sgemv = FISTA_module.get_function("Sgemv")
soft_thresh_S = FISTA_module.get_function("soft_thresh_S")

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float32)
tr_beta = np.zeros(p).astype(np.float32)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float32)


def FISTA_S(beta, X, y, lam, L, eta, tol = 1e-04, max_iter = 5000):
    n = np.int32(X.shape[0])
    p = np.int32(X.shape[1])
    t = np.float32(1.0)
    One = np.float32(1.0)
    MOne = np.float32(-1.0)
    IntOne = np.int32(1)
    IntZero = np.int32(0)
    
    crit = np.zeros(max_iter, dtype=np.float32)
    temp = np.zeros(p, dtype=np.float32)
    d_beta_p = gpuarray.to_gpu(temp)
    d_beta_prev = gpuarray.to_gpu(temp)
    d_X = gpuarray.to_gpu(X)
    d_y = gpuarray.to_gpu(y)
    d_ymXbp = gpuarray.empty(n, dtype = np.float32)
    d_beta = gpuarray.empty(p, dtype = np.float32)
  
    TPB = (32, 1, 1)
    bpg_p = math.ceil(np.float32(p)/TPB[0])
    bpg_n = math.ceil(np.float32(n)/TPB[0])
    BPG_p = (bpg_p, 1, 1)
    BPG_n = (bpg_n, 1, 1)
  
    L_prev = L
    for k in range(max_iter):
        d_ymXbp = d_y.copy()
        Sgemv(MOne, d_X, d_beta_p, 
              d_ymXbp, n, p, IntZero,
              grid = BPG_n, block = TPB)
        h_rbp = gpuarray.dot(d_ymXbp, d_ymXbp)
        d_XTrbp = gpuarray.zeros(p, np.float32)
        Sgemv(One, d_X, d_ymXbp, d_XTrbp, n, p,
              IntOne, grid = BPG_p, block = TPB)
        
        i_k = -1
        cond = True
        while cond:
            i_k += 1
            eta_ik = eta ** i_k
            L_cur = L_prev * eta_ik
            alpha = np.float32(1.0/L_cur)  
            d_bstar = d_beta_p + alpha*d_XTrbp
            alpha = np.float32(lam/L_cur)
            soft_thresh_S(d_bstar, alpha, d_beta, p,
                         grid = BPG_p, block = TPB)
            d_diff_beta = d_beta - d_beta_p
            h_RHS_1st = gpuarray.dot(d_diff_beta,
                                     d_diff_beta)
            h_RHS_2nd = gpuarray.dot(d_diff_beta,
                                     d_XTrbp)
              
            RHS1 = L_cur * h_RHS_1st.get()
            RHS2 = np.float32(2.0)*h_RHS_2nd.get()
            RHS =  RHS1 - RHS2 
                              
            d_ymXb = d_y.copy()
            Sgemv(MOne, d_X, d_beta, d_ymXb, n, p, 
                  IntZero, grid = BPG_n, block = TPB)
            d_ymXb2 = gpuarray.dot(d_ymXb, d_ymXb)
            LHS = d_ymXb2.get() - h_rbp.get()
            cond = (LHS > RHS)
          
        L_prev = L_cur
        tnext = np.float32((1.0+np.sqrt(1+4*t*t))/2)
        d_diff_beta = d_beta - d_beta_prev
        alpha = np.float32((t - 1.0)/tnext)
        d_beta_p = d_beta + alpha * d_diff_beta
        d_diff_b_sq = gpuarray.dot(d_diff_beta,
                                   d_diff_beta)
        crit[k] = np.sqrt(d_diff_b_sq.get())
          
        if crit[k] < tol:
            break
          
        t = tnext
        d_beta_prev = d_beta.copy()
          
    return d_beta.get(), crit, k

for i in np.arange(niter):

    beta = np.zeros(p, dtype = np.float32)
    lam = np.sqrt(2*np.log(p)/n).astype(np.float32)
    L = np.float32(10)
    eta = np.float32(2)
    
    t1 = time.time()
    out, crit, step = FISTA_S(beta, X, y, lam, L, eta, tol = 1e-3, max_iter = 5000)
    t2 = time.time()
    comp_time_fista[i,2] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,2])




  1 -th iteration, Collapsed Time:  4.30243182182312

  2 -th iteration, Collapsed Time:  3.577413558959961

  3 -th iteration, Collapsed Time:  3.624744415283203

  4 -th iteration, Collapsed Time:  3.655689239501953

  5 -th iteration, Collapsed Time:  3.7542014122009277

  6 -th iteration, Collapsed Time:  3.6771984100341797

  7 -th iteration, Collapsed Time:  3.700299024581909

  8 -th iteration, Collapsed Time:  3.5761942863464355

  9 -th iteration, Collapsed Time:  3.650284767150879

  10 -th iteration, Collapsed Time:  3.4707753658294678

  11 -th iteration, Collapsed Time:  3.5973548889160156


In [11]:
step

1742

In [12]:
out[:5]

array([0.9498172 , 0.97780025, 0.94975954, 0.9497829 , 1.0544457 ],
      dtype=float32)

In [13]:
import numpy as np
from numba import cuda
import math
import time

@cuda.jit
def gemv(alpha, A, x, y, tran):
    Row = cuda.grid(1)
    n = A.shape[0]
    p = A.shape[1]
    pvalue = 0.
    if tran:
        if Row < p:
            for i in range(n):
                pvalue += A[i, Row] * x[i]
            y[Row] += alpha * pvalue
    else:
        if Row < n:
            for j in range(p):
                pvalue += A[Row, j] * x[j]
            y[Row] += alpha * pvalue

@cuda.jit
def soft_thr(x, alpha, S):
    idx = cuda.grid(1)
    if idx < x.shape[0]:
        thr = math.fabs(x[idx]) - alpha
        if thr > 0:
            S[idx] = math.copysign(thr, x[idx])
        else:
            S[idx] = 0

@cuda.jit
def axpy(alpha, x, y):
    idx = cuda.grid(1)
    if idx < x.shape[0]:
        y[idx] += alpha * x[idx]

        
@cuda.jit
def vec_prod(x, y, res):
    idx = cuda.grid(1)
    if idx < x.shape[0]:
        res[idx] = x[idx] * y[idx]

@cuda.reduce
def reduce(x, y):
    return x+y

def FISTA(beta, X, y, lamb, L = 10, eta = 1.2, tol = 1e-08, 
          max_iter = 5000, dtype = np.float32):
    n = X.shape[0]
    p = X.shape[1]
    t = dtype(1.0)
    crit = np.zeros(max_iter, dtype = dtype)
    d_bstar = np.zeros(p, dtype = dtype)
    d_beta = np.zeros(p, dtype = dtype)
    d_diff_beta2 = np.zeros(p, dtype = dtype)
    d_XTrbpd = np.zeros(p, dtype = dtype)
    d_beta_prev = np.zeros(p,dtype = dtype)
    d_beta_p = np.zeros(p, dtype = dtype)
    d_ymXbp2 = np.zeros(n, dtype = dtype)
    d_ymXb2= np.zeros(n, dtype = dtype)
    L_prev = L
    TPB = (32,1)
    BPG_p = (math.ceil(p / 32) , 1)
    BPG_n = (math.ceil(n / 32) , 1)
    
    for k in range(max_iter):
        d_ymXbp = y.copy()
        gemv[BPG_n, TPB](-1.0, X, d_beta_p, d_ymXbp, False)
        vec_prod[BPG_n, TPB](d_ymXbp, d_ymXbp, d_ymXbp2)
        h_rbp = reduce(d_ymXbp2)
        d_XTrbp = np.zeros(p, dtype = dtype)
        gemv[BPG_p, TPB](1.0, X, d_ymXbp, d_XTrbp, True)
        
        i_k = -1
        cond = True
        while cond:
            i_k += 1
            L_cur = L_prev * (eta ** i_k)
            d_bstar = d_beta_p.copy()
            axpy[BPG_p, TPB]((1.0 / L_cur), d_XTrbp, d_bstar)
            soft_thr[BPG_p, TPB](d_bstar, lamb / L_cur, d_beta)
            d_diff_beta = d_beta.copy()
            axpy[BPG_p, TPB](-1.0, d_beta_p, d_diff_beta)
            vec_prod[BPG_p, TPB](d_diff_beta, d_diff_beta, d_diff_beta2)
            vec_prod[BPG_p, TPB](d_XTrbp, d_diff_beta, d_XTrbpd)
            RHS = L_cur * reduce(d_diff_beta2) - 2.0 * reduce(d_XTrbpd)
            d_ymXb = y.copy()
            gemv[BPG_n, TPB](-1.0, X, d_beta, d_ymXb, False)
            vec_prod[BPG_n, TPB](d_ymXb, d_ymXb, d_ymXb2)
            h_rb = reduce(d_ymXb2)
            LHS =  h_rb - h_rbp
            cond = (LHS > RHS)
        
        L_prev = L_cur
        tnext = (1.0 + np.sqrt(1 + 4 * t**2) ) / 2.0
        d_diff_beta = d_beta.copy()
        axpy[BPG_p, TPB](-1.0, d_beta_prev, d_diff_beta)
        t1 = (t - 1.0) / tnext
        d_beta_p = d_beta.copy()
        axpy[BPG_p, TPB](t1, d_diff_beta, d_beta_p)
        vec_prod[BPG_p, TPB](d_diff_beta, d_diff_beta, d_diff_beta2)
        crit[k] = np.sqrt(reduce(d_diff_beta2))
        if crit[k] < tol:
            break
        t = tnext
        d_beta_prev = d_beta.copy()
    return d_beta, crit, k

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float32)
tr_beta = np.zeros(p).astype(np.float32)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float32)

cuda.select_device(0)
for i in np.arange(niter):

    beta = np.zeros(p, dtype = np.float32)
    lam = np.sqrt(2*np.log(p)/n).astype(np.float32)
    L = np.float32(10)
    eta = np.float32(2)
    
    t1 = time.time()
    out, crit, step = FISTA(beta, X, y, lam, L, eta, tol = 1e-03, max_iter = 5000)
    t2 = time.time()
    comp_time_fista[i,3] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,3])
    



  1 -th iteration, Collapsed Time:  79.7794075012207

  2 -th iteration, Collapsed Time:  78.54222583770752

  3 -th iteration, Collapsed Time:  78.82252240180969

  4 -th iteration, Collapsed Time:  79.10402488708496

  5 -th iteration, Collapsed Time:  79.04960012435913

  6 -th iteration, Collapsed Time:  79.51575303077698

  7 -th iteration, Collapsed Time:  78.22756361961365

  8 -th iteration, Collapsed Time:  79.01998257637024

  9 -th iteration, Collapsed Time:  79.4259901046753

  10 -th iteration, Collapsed Time:  79.55859065055847

  11 -th iteration, Collapsed Time:  78.1091775894165


In [14]:
step

1733

In [15]:
out[:5]

array([0.95338017, 0.9755957 , 0.9560744 , 0.9505501 , 1.0496136 ],
      dtype=float32)

In [16]:
import torch
import numpy as np
import time
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.init()
torch.backends.cuda.matmul.allow_tf32 = False

def soft_thr(x, alpha):
    n = x.shape[0]
    S = torch.maximum(torch.abs(x)-alpha, 
        torch.zeros(n,device='cuda:0'))*torch.sign(x)
    return S

def FISTA(beta, X, y, lamb, L, eta , tol = 1e-04, max_iter = 5000, dtype = torch.float32):
    if(dtype == torch.float32):
        torch.set_default_tensor_type(torch.FloatTensor)
    else:
        torch.set_default_tensor_type(torch.DoubleTensor)
    device = torch.device("cuda:0")
    n = X.shape[0]
    p = X.shape[1]
    dbeta = torch.Tensor(beta).to(device)
    dX = torch.Tensor(X).to(device)
    dy = torch.Tensor(y).to(device)
    t = torch.ones(1,dtype=dtype, device=device)
    crit = np.zeros(max_iter)
    dbeta_p = torch.Tensor(beta).to(device)
    dbeta_prev = torch.Tensor(beta).to(device)
    L_prev = L
    for k in range(max_iter):
        dymXbp = dy-torch.matmul(dX,dbeta_p)
        drbp = torch.dot(dymXbp, dymXbp)
        dXTrbp = torch.matmul(torch.t(dX),dymXbp)

        i_k = -1
        cond = True
        while cond:
            i_k += 1
            L_cur = L_prev*(eta**i_k)
            dbstar = dbeta_p + dXTrbp/L_cur
            dbeta = soft_thr(dbstar, lamb/L_cur)
            diff_beta = dbeta - dbeta_p
            RHS_1st = torch.dot(diff_beta,diff_beta)
            RHS_2nd = torch.dot(diff_beta,dXTrbp)
            RHS = L_cur*RHS_1st-2.0*RHS_2nd
            dymXb = dy-torch.matmul(dX,dbeta)
            LHS = torch.dot(dymXb,dymXb)-drbp
            cond = (LHS>RHS)
        L_prev = L_cur
        tnext = (1.0+torch.sqrt(1+4*t**2))/2.0
        diff_beta = dbeta-dbeta_prev
        t1 = (t-1.0)/tnext
        dbeta_p = dbeta+t1*diff_beta
        crit[k] = torch.linalg.norm(diff_beta).to('cpu')
        if crit[k] < tol:
            break
        t = tnext
        dbeta_prev = dbeta
    out = dbeta.to('cpu')
    return out.numpy(), crit, k

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float32)
tr_beta = np.zeros(p).astype(np.float32)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float32)


for i in np.arange(niter):

    beta = np.zeros(p, dtype = np.float32)
    lam = np.sqrt(2*np.log(p)/n).astype(np.float32)
    L = np.float32(10)
    eta = np.float32(2)
    
    t1 = time.time()
    out,crit,step = FISTA(beta, X, y, lam, L, eta, tol = 1e-03, max_iter = 5000)
    t2 = time.time()
    comp_time_fista[i,5] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,5])
   




  1 -th iteration, Collapsed Time:  2.9106998443603516

  2 -th iteration, Collapsed Time:  0.996375322341919

  3 -th iteration, Collapsed Time:  1.0285370349884033

  4 -th iteration, Collapsed Time:  0.9759187698364258

  5 -th iteration, Collapsed Time:  1.011160135269165

  6 -th iteration, Collapsed Time:  0.9376533031463623

  7 -th iteration, Collapsed Time:  0.9548244476318359

  8 -th iteration, Collapsed Time:  0.9287333488464355

  9 -th iteration, Collapsed Time:  0.9455447196960449

  10 -th iteration, Collapsed Time:  0.9345541000366211

  11 -th iteration, Collapsed Time:  0.9375431537628174


In [17]:
step

1729

In [18]:
out[:5]

array([0.9568989 , 0.9750053 , 0.95734704, 0.94645965, 1.0518653 ],
      dtype=float32)

In [19]:
crit[:9]

array([3.14292955, 1.54338944, 1.21923745, 0.96647245, 0.78125411,
       0.64401644, 0.53842074, 0.45384514, 0.38440982])

In [20]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import tensorflow as tf
import numpy as np
import time

@tf.function
def soft_thr(x,alpha):
    n = x.shape[0]
    S = tf.math.maximum(tf.abs(x)-alpha,0)*tf.math.sign(x)
    return S

def FISTA(beta, X, y, lamb, L, eta, tol = 1e-08, max_iter = 5000, dtype = tf.float32):
    n = X.shape[0]
    p = X.shape[1]
    crit = np.zeros(max_iter)
  
    with tf.device("/GPU:0"):
        dbeta = tf.constant(beta)
        dX = tf.constant(X)
        dy = tf.constant(y)
        t = tf.constant(1.0, dtype=dtype)
        dbeta_p = tf.constant(beta)
        dbeta_prev = tf.constant(beta)
        L_prev = tf.constant(L,dtype=dtype)

        for k in range(max_iter):
            dymXbp = dy-tf.linalg.matvec(dX,dbeta_p)
            drbp = tf.tensordot(dymXbp,dymXbp,axes=1)
            dXTrbp = tf.linalg.matvec(dX,dymXbp, 
                                    transpose_a=True)

            i_k = -1
            cond = True
            while cond:
                i_k += 1
                L_cur = L_prev*(eta**i_k)
                dbstar = dbeta_p + dXTrbp/L_cur
                dbeta = soft_thr(dbstar, lamb/L_cur)
                diff_beta = dbeta - dbeta_p
                RHS_1st = tf.tensordot(diff_beta, 
                                       diff_beta,axes=1)
                RHS_2nd = tf.tensordot(diff_beta, dXTrbp,
                                       axes=1)
                RHS = L_cur * RHS_1st.numpy() - 2.0 * RHS_2nd.numpy()
                dymXb = dy - tf.linalg.matvec(dX,dbeta)
                LHS = (tf.tensordot(dymXb,dymXb,
                       axes=1)).numpy()-drbp.numpy()
                cond = (LHS>RHS)
            L_prev = L_cur
            tnext =  (1.0+tf.sqrt(1+4*t**2))/2.0
            diff_beta = dbeta-dbeta_prev
            t1 = (t - 1.0)/tnext
            dbeta_p = dbeta+t1*diff_beta
            crit[k] = tf.norm(diff_beta) 
            if crit[k] < tol:
                break
            t = tnext
            dbeta_prev = dbeta
    return dbeta.numpy(), crit, k

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float32)
tr_beta = np.zeros(p).astype(np.float32)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float32)

for i in np.arange(niter):

    beta = np.zeros(p, dtype = np.float32)
    lam = np.sqrt(2*np.log(p)/n).astype(np.float32)
    L = np.float32(10)
    eta = np.float32(2)
    
    t1 = time.time()
    out, crit, step = FISTA(beta, X, y, lam, L, eta, tol = 1e-03, max_iter = 5000)
    t2 = time.time()
    comp_time_fista[i,4] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,4])
   



  1 -th iteration, Collapsed Time:  7.934950113296509

  2 -th iteration, Collapsed Time:  7.2251622676849365

  3 -th iteration, Collapsed Time:  6.943118333816528


In [ ]:
step

In [ ]:
out[:5]

In [ ]:
with open('./fista_comp_time_n{}_p{}_float.txt'.format(n,p), 'w') as f:
    print(('%s\t '*6 % ('DLL-K','DLL-cuBLAS','PyCUDA','Numba','TF','PyTorch')), end='\n', file=f)
    for i in range(niter):
        print(('%.4f\t '*6 % tuple(comp_time_fista[i])), end='\n', file=f)
